# CS4248 Assignment 3

### Perceptron text classifier in Python with feature selection

##### https://github.com/jia1/pyceptron

In [1]:
import os
import re
import sys
from string import punctuation
from porter import PorterStemmer

p = PorterStemmer()

In [2]:
src_dir = os.path.abspath('tc')

stopword_list_file = 'stopword-list'
train_class_list_file = 'train-class-list'
model_file = 'model'

In [3]:
'''
class_to_text
<class 'dict'>
KEY: <class 'str'>
VAL: <class 'set'>
E.g: {'c1': {'0000', '0001', '0002', ... }, 'c2': {'1000', '1001', '1002', ... } }

text_to_freq
<class 'dict'>
KEY: <class 'str'>
VAL: <class 'dict'>
     KEY: <class 'str'>
     VAL: <class 'int'>
E.g: {'0000': {'word0': 1, 'word1': 3, 'word2': 5, ... }, '0001': { ... }}

class_to_feat_chi_tup
<class 'dict'>
KEY: <class 'str'>
VAL: <class 'list'>
     VAL: <class 'tuple'>
          <class 'str'>
          <class 'float'>
E.g: {'c1': [ ('word1', 99.00), ('word2', 90.00), ... ], 'c2': [ ... ] }

nxx_dict
chi_dict
'''

k = 3
max_compromise = 1
num_both, num_train = 0, 0
train_ratio = 1
# train_ratio = 0.8
# test_ratio = 1 - train_ratio

num_class = 0
class_list = []
class_to_text, text_to_freq = {}, {}

In [4]:
class_to_feat_chi_tup = {}
class_to_feat_set = {}
class_to_feat_list_sort_by_chi = {}

feat_prune_ratio = 0.5

nxxs = ['n10', 'n11']
nxxs_map = {
    'n00': 'n10',
    'n01': 'n11'
}
nxx_dict = { n: {} for n in nxxs }
chi_dict = {}

In [5]:
# A. Split each line by space into tokens
# B. Strip all default white space characters from each token
# C. Remove punctuation from each token
# D. Return a list of tokens which are not stop words

def strip_and_filter_line(ln):
    if all(x in ln for x in [':', '@']):
        return []
    tokens = map(lambda t: t.strip().strip(punctuation).lower(), ln.split(' '))
    return list(filter(lambda t: t and len(t) > 2 and t.isalpha() and t not in stop_list, tokens))

In [6]:
def is_in(a, b):
    return 1 if a in b else 0

def count_nxx(nxx, w, c):
    global class_list, class_to_text, text_to_freq
    answer = 0
    if nxx == 'n10':
        for class_name in filter(lambda x: x != c, class_list):
            for text in class_to_text[class_name]:
                answer += is_in(w, text_to_freq[text])
    elif nxx == 'n11':
        for text in class_to_text[c]:
            answer += is_in(w, text_to_freq[text])
    return answer

In [7]:
# n00 is the number of training texts that do not contain w and are not in class c.
# n01 is the number of training texts that do not contain w and are in class c.
# n10 is the number of training texts that contain w and are not in class c.
# n11 is the number of training texts that contain w and are in class c.

# n00 = num_train - n10
# n01 = num_train - n11
# n10 = num_train - n00
# n11 = num_train - n01

def chi_square(w, c):
    global num_train, nxxs, nxxs_map, nxx_dict
    ns_dict = {}
    for n in nxxs:
        if w not in nxx_dict[n]:
            nxx_dict[n][w] = {}
        if c not in nxx_dict[n][w]:
            nxx_dict[n][w][c] = count_nxx(n, w, c)
        ns_dict[n] = nxx_dict[n][w][c]
    for n, nn in nxxs_map.items():
        ns_dict[n] = num_train - nxx_dict[nn][w][c]
    n00, n01, n10, n11 = ns_dict['n00'], ns_dict['n01'], ns_dict['n10'], ns_dict['n11']
    return ((n11+n10+n01+n00)*(n11*n00-n10*n01)**2)/((n11+n01)*(n11+n10)*(n10+n00)*(n01+n00))

In [8]:
def put_chi_dict(c, w, chi_square_value):
    global chi_dict
    if w not in chi_dict[c]:
        chi_dict[c][w] = chi_square_value
    else:
        chi_dict[c][w] = max(chi_dict[c][w], chi_square_value)

In [9]:
def gen_feat():
    global class_list, chi_dict, class_to_feat_chi_tup
    max_feat_len = sys.maxsize
    feat_queue_dict = { c: [] for c in class_list }
    for c in chi_dict:
        feat_queue_dict[c] = sorted(chi_dict[c].items(), key = lambda x: x[1], reverse = True)
        max_feat_len = min(max_feat_len, len(feat_queue_dict[c]))
    max_feat_len *= feat_prune_ratio 
    class_to_feat_chi_tup = { c: feat_queue_dict[c][:int(max_feat_len)] for c in feat_queue_dict }

In [10]:
def feat_select():
    global class_list, class_to_text, text_to_freq, class_to_feat_chi_tup
    for c in class_list:
        for text in class_to_text[c]:
            for w in text_to_freq[text]:
                put_chi_dict(c, w, chi_square(w, c))
                gen_feat()

In [11]:
with open(stopword_list_file, 'r') as s:
    stop_list = list(map(lambda ln: ln.strip(), s.readlines()))

In [ ]:
with open(train_class_list_file, 'r') as t:
    lines = map(lambda ln: ln.strip().split(' '), t.readlines())
    for ln in lines:
        file, curr_class = ln
        text = file.split('/')[-1]
        num_both += 1
        num_train += 1
        flat_text, freq_dict = [], {}
        with open(file, 'r') as f:
            processed_lines = map(lambda ln: strip_and_filter_line(ln), f.readlines())
            for line in processed_lines:
                flat_text.extend(list(map(lambda word: p.stem(word, 0, len(word) - 1), line)))
            num_words = len(flat_text)
            last_index = num_words - 1
            for i in range(num_words):
                word = flat_text[i]
                if word not in freq_dict:
                    freq_dict[word] = 1
                else:
                    freq_dict[word] += 1
                if i < last_index:
                    bigram = '{} {}'.format(word, flat_text[i + 1])
                    if bigram not in freq_dict:
                        freq_dict[bigram] = 1
                    else:
                        freq_dict[bigram] += 1
            fin_freq_dict = { word: freq for word, freq in freq_dict.items() if freq >= k }
            compromise = 1
            while not fin_freq_dict and compromise <= max_compromise:
                fin_freq_dict = { word: freq for word, freq in freq_dict.items() if freq >= k - compromise }
                compromise += 1
            len_gram = 2
            while not fin_freq_dict and len_gram:
                fin_freq_dict = { word: freq for word, freq in freq_dict.items() if len(word.split(' ')) >= len_gram }
                len_gram -= 1
            sum_freq = sum(fin_freq_dict.values())
            if curr_class not in class_list:
                class_list.append(curr_class)
            if curr_class not in class_to_text:
                class_to_text[curr_class] = set()
            else:
                class_to_text[curr_class].add(text)
            if text not in text_to_freq:
                text_to_freq[text] = {}
            else:
                text_to_freq[text] = { word: freq / sum_freq for word, freq in fin_freq_dict.items() }

In [12]:
for curr_dir, sub_dir, files in os.walk(src_dir):
    if not files:
        class_list = sub_dir
        num_class = len(class_list)
        class_to_text = { c: set() for c in class_list }
        chi_dict = { c: {} for c in class_list }
        class_to_feat_chi_tup = { c: set() for c in class_list }
        class_to_feat_set = { c: set() for c in class_list }
        class_to_feat_to_index = { c: {} for c in class_list }
        continue
    curr_class = re.split('[(\\\\)(\\)(\/)]', curr_dir)[-1] # curr_dir.split('\\')[-1]
    curr_num_files = len(files)
    num_both += curr_num_files
    curr_num_train = int(curr_num_files * train_ratio)
    num_train += curr_num_train
    for i in range(curr_num_train):
        file = files[i]
        flat_text, freq_dict = [], {}
        with open(os.path.join(curr_dir, file), 'r') as f:
            processed_lines = map(lambda ln: strip_and_filter_line(ln), f.readlines())
            for line in processed_lines:
                flat_text.extend(list(map(lambda word: p.stem(word, 0, len(word) - 1), line)))
            num_words = len(flat_text)
            last_index = num_words - 1
            for i in range(num_words):
                word = flat_text[i]
                if word not in freq_dict:
                    freq_dict[word] = 1
                else:
                    freq_dict[word] += 1
                if i < last_index:
                    bigram = '{} {}'.format(word, flat_text[i + 1])
                    if bigram not in freq_dict:
                        freq_dict[bigram] = 1
                    else:
                        freq_dict[bigram] += 1
            fin_freq_dict = { word: freq for word, freq in freq_dict.items() if freq >= k }
            compromise = 1
            while not fin_freq_dict and compromise <= max_compromise:
                fin_freq_dict = { word: freq for word, freq in freq_dict.items() if freq >= k - compromise }
                compromise += 1
            len_gram = 2
            while not fin_freq_dict and len_gram:
                fin_freq_dict = { word: freq for word, freq in freq_dict.items() if len(word.split(' ')) >= len_gram }
                len_gram -= 1
            sum_freq = sum(fin_freq_dict.values())
            class_to_text[curr_class].add(file)
            text_to_freq[file] = { word: freq / sum_freq for word, freq in fin_freq_dict.items() }

In [ ]:
import pprint
pp = pprint.PrettyPrinter(indent = 4)
pp.pprint(text_to_freq)

In [13]:
from time import time
start = time()

chi_dict = { c: {} for c in class_list }
class_to_feat_chi_tup = { c: set() for c in class_list }
feat_select()

stop = time()
print(stop - start)

48.88188815116882


In [ ]:
import pprint
pp = pprint.PrettyPrinter(indent = 4)
pp.pprint(class_to_feat_chi_tup)

In [14]:
class_to_feat_set = { c: set() for c in class_list }

for c in class_to_feat_chi_tup:
    for p in class_to_feat_chi_tup[c]:
        w = p[0]
        class_to_feat_set[c].add(w)
    curr_num_feat = len(class_to_feat_set[c])
    num_feat_per_neg_class = curr_num_feat // (num_class - 1)
    for nc in class_to_feat_chi_tup:
        if nc != c:
            num_added = 0
            for t in class_to_feat_chi_tup:
                class_to_feat_set[c].add(t[0])
                if num_added >= num_feat_per_neg_class:
                    break

class_to_feat_list_sort_by_chi = { c: sorted(list(class_to_feat_set[c])) for c in class_list }
class_to_feat_to_index = { c: {} for c in class_list }

for c in class_to_feat_list_sort_by_chi:
    for i in range(len(class_to_feat_list_sort_by_chi[c])):
        class_to_feat_to_index[c][class_to_feat_list_sort_by_chi[c][i]] = i

In [ ]:
import pprint
pp = pprint.PrettyPrinter(indent = 4)
# pp.pprint(class_to_feat_to_index)
print(class_to_feat_to_index)

In [15]:
# Credits: Dr. Jason Brownlee
from random import seed, randrange
seed(4248)

# Split data_mat into num_folds number of folds
def get_folds(data_mat, num_folds):
    folds = []
    data_clone = list(data_mat)
    fold_size = int(len(data_mat) / num_folds)
    for i in range(num_folds):
        fold = []
        while len(fold) < fold_size:
            index = randrange(len(data_clone))
            fold.append(data_clone.pop(index))
        folds.append(fold)
    return folds

# Calculate accuracy percentage
def get_accuracy(predicted, actual):
    num_correct = 0
    for i in range(len(actual)):
        if predicted[i] == actual[i]:
            num_correct += 1
    return num_correct / len(actual) * 100

# Evaluate an algorithm using a cross validation split
def get_cross_validation_scores(data_mat, algorithm, num_folds, *args):
    folds = get_folds(data_mat, num_folds)
    scores = []
    for fold in folds:
        train_set = list(folds)
        train_set.remove(fold)
        train_set = [row for fold in train_set for row in fold]
        test_set = []
        for row in fold:
            row_clone = list(row)
            test_set.append(row_clone)
            row_clone[-1] = None
        predicted = algorithm(train_set, test_set, *args)
        actual = [row[-1] for row in fold]
        accuracy = get_accuracy(predicted, actual)
        scores.append(accuracy)
    return scores

# Make a prediction with weights
def predict(row, weights):
    activation = weights[0]
    for i in range(len(row) - 1):
        activation += weights[i + 1] * row[i]
    return 1 if activation >= 0 else 0

# Estimate Perceptron weights using stochastic gradient descent
def train_weights(train, alpha, max_iterations = 1000):
    weights = [0 for i in range(len(train[0]))]
    for _ in range(max_iterations):
        for row in train:
            prediction = predict(row, weights)
            error = row[-1] - prediction
            weights[0] = weights[0] + alpha * error
            for i in range(len(row) - 1):
                weights[i + 1] += alpha * error * row[i]
    return weights

# Perceptron Algorithm With Stochastic Gradient Descent
def perceptron(train, test, alpha, max_iterations):
    predictions = list()
    weights = train_weights(train, alpha, max_iterations)
    print(weights)
    for row in test:
        prediction = predict(row, weights)
        predictions.append(prediction)
    return predictions

In [16]:
# load and prepare data
class_to_feat_mat = { c: [] for c in class_list }
for c in class_list:
    for d in class_list:
        texts = class_to_text[d]
        num_texts = len(texts)
        texts = iter(texts)
        if c != d:
            num_texts_to_train = int(num_texts * train_ratio / (num_class - 1))
        else:
            num_texts_to_train = num_texts
        for i in range(num_texts_to_train):
            text = next(texts)
            feat_vec = [0 for i in range(len(class_to_feat_to_index[d]) + 1)]
            for word in text_to_freq[text]:
                if word in class_to_feat_to_index[d]:
                    index = class_to_feat_to_index[d][word]
                    feat_vec[index] = text_to_freq[text][word]
            feat_vec[-1] = 1 if c == d else 0
            class_to_feat_mat[c].append(feat_vec)

In [ ]:
data = class_to_feat_mat
num_folds_list = [5, 10] # [5, 10]
alpha_list = [0.05, 0.1] # [0.02, 0.03, 0.05, 0.07, 0.1]
max_iterations_list = [500, 1000, 2000] # [500, 1000, 2000]

for num_folds in num_folds_list:
    for alpha in alpha_list:
        for max_iterations in max_iterations_list:
            print('{}-fold cross validation'.format(num_folds))
            print('Learning rate: {}, maximum number of iterations: {}'.format(alpha, max_iterations))
            for c in class_list:
                scores = get_cross_validation_scores(data[c], perceptron, num_folds, alpha, max_iterations)
                print('Class: {}'.format(c))
                print('Cross validation scores: {}'.format(scores))
                print('Mean accuracy: {:.2f}%'.format(sum(scores) / num_folds))
                print()

5-fold cross validation
Learning rate: 0.05, maximum number of iterations: 500
[-1.3877787807814457e-17, -0.025423728813559327, -0.13366637241750434, -0.03089566636984272, 0.11206248050153962, 0.3761692683197338, 0.163074036382139, -0.19380962915832484, 0.10526261629988772, -0.26370123926207767, -0.1518824540730778, -0.2630409868649789, 0.39417536220617005, 0.017316725268149424, -0.13215453735934626, -0.4029067184302211, 0.02911193035196988, 0.03831826122700628, -0.049283284140414316, 0.06078451181735151, -0.007546749777381997, -0.05054945054945054, 0.075, -0.007993884080254978, -0.05390624999999998, 0.24708035846643217, -0.16404589064587793, -0.10355029585798824, -0.08925448846802743, -0.08692922123532633, 0.39517865392455515, -0.10606060606060605, -0.03875111746730209, -0.1552898589703724, -0.041666666666666664, 0.056298076923076916, -0.011315252881128416, -0.06925492529678579, -0.1711220606523971, -0.007359859714605882, -0.5274107552889421, -0.04879165659218079, -0.02674649488487532

[-1.3877787807814457e-17, 0.007520275251904615, -0.1419418651889836, 0.00489072588089923, 0.0658506473283797, 0.40749797664262205, -0.020669612748566973, -0.09364107373139921, -0.1319585034186811, -0.19415716096324456, -0.04249772907286554, -0.20983913222595096, 0.1492194318691979, 0.03243291241336448, -0.022620299391423786, -0.27509875142854195, -0.012646326666870716, 0.1607011954920685, -0.5937239542740218, 0.0022132869991253606, 0.013713125528881601, 0.0708715941592654, 0.1465454532116553, -0.05559135682449042, -0.018303571428571433, 0.12983549283345638, -0.20761537963906374, -0.13402366863905338, -0.057598750333001984, -0.004423047422540402, 0.2656450163322548, -0.11183951830916772, -0.05033484095389571, 0.056572489876386974, -0.04976693781361284, 0.12475024975024948, -0.013713362686551376, -0.11297430913831391, -0.10846594933909448, -0.08538001080825486, -0.6555527539557006, -0.13454365030547794, -0.06164794042830388, 0.11249999999999999, -0.07826086956521743, 0.04602587336638855,

[-1.3877787807814457e-17, 0.0032084258406887903, -0.10384538830131085, -0.0249092221220164, 0.14092436276373577, -0.014277763037254778, 0.12151649588901003, -0.1517334116921586, 0.07286714116099949, -0.4245004224757096, -0.045034201093334864, -0.33314886436944247, 0.227126129619876, -0.03389307162395921, -0.0482802883897673, -0.25150833884868684, 0.3181412293138526, -0.31059072380365127, -0.16749364652805188, 0.016465443940540517, 0.0062424346176261445, 0.030769230769230785, 0.19154545321165528, -0.06189993935472904, 0.0, -0.003868864902773063, -0.33496060904896524, -0.07248520710059193, -0.014310543863472872, -0.2926956382414611, 0.3990937524487544, -0.15993610223642232, -0.03615787818113349, -0.33555915546815546, -0.010665198232640009, 0.14802697302697274, -0.14500337626791363, -0.04914154860798245, -0.20501427638067077, -0.02581399564555269, -0.45530222356096167, -0.027830124702122733, 0.19949371099657218, 0.11249999999999999, -0.20625000000000016, -0.08875839998319729, -0.055188679

[-1.3877787807814457e-17, 0.002913513085712138, -0.10086729298678863, 0.008381021035111014, 0.04388445748633061, 0.5090730602265524, 0.0790600654094273, -0.25686447696071135, 0.07793054839501128, -0.23324074074074078, -0.24318014254116452, -0.3476312504630217, 0.3792621964035512, -0.054644100657952076, -0.05987338511198248, -0.2677355928236812, 0.17612805204499266, 0.04202702702702745, -0.40957964989928897, 0.08139937486210556, 0.005564995333960385, 0.026373626373626384, 0.14796054755127794, 0.005725067004286904, -0.10887706043956055, 0.20921461546367406, 0.3219660952006743, -0.10192307692307696, 0.034474522292993665, -0.14960573476702502, 0.10392394684787218, 0.009547800442369136, -0.11405575724364672, -0.18661055344906619, -0.037789526767162615, 0.048038836163836154, -0.08117636718333338, 0.01356750865288919, -0.0888298602350624, -0.04424899817601126, -1.0028449619965851, 0.003315348534574069, -0.017516365311416437, 0.11249999999999999, -0.5173295454545446, 0.06978260869565218, -0.05

[0.0, 0.0064389284836568975, -0.05662924418171277, 0.00482919636274268, 0.037778495318049735, 0.5050788591078474, 0.1569095756498556, -0.1994662596785637, 0.1307938905480057, -0.30055555555555635, -0.21981175129474229, -0.11866520738999416, 0.35699265418146864, -0.054077760497667174, 0.019406392694063926, -0.3415875952414562, 0.12948035421096254, -0.22500000000000048, -0.06110931726043202, 0.13567918633604503, 0.01755153233472215, 0.038461538461538464, -0.02269982980721265, -0.004688084160429146, -0.008816964285714287, 0.10759892507838048, -0.1818702487071736, -0.07500000000000001, -0.015000000000000041, -0.026666666666666672, 0.34284610497024964, -0.11373187515360103, -0.06153846153846154, -0.20205277623827272, 0.009631103720560578, 0.07886925574425568, 0.051502202428909565, 0.08928571428571427, -0.2850344704543772, 0.03, -0.26720871695816345, -0.015532078433895027, 0.06875318070898599, 0.037500000000000006, -0.3707880434782603, 0.052335630965005306, -0.06792452830188679, 0.1468860003

...

[0.0, 0.022881355932203393, 0.07512329611617231, 0.12433456174398645, 0.06484440267335001, 0.29746757454193373, -0.019338166385563987, -0.3206515596036538, 0.47599313617806327, 0.004381551362683325, 0.24492753623188365, 0.11674418604651166, -0.008542620480438497, 0.5577029930626153, -0.014567209861968366, -0.16880385090779654, 0.007130333138515506, -0.19798558347944664, -0.17026183748133944, 0.2821999049927433, -0.0005117528350772689, 0.32156593406593453, -0.007553275197593809, -0.08554361710385656, -0.05570522583197831, 0.00410272850806134, 0.14288483221638798, -0.025000000000000005, 0.01703821656050871, 0.22580645161290308, 0.06415012945798747, 0.045018369089723634, 0.05311428182414366, -0.11988975109188799, 0.10837140233065748, 0.09483870967741939, 0.1129597598080013, -0.07580011523754655, 0.2938378100704521, -0.04098719382624302, 0.03302341417421826, 0.035499216571834066, -0.03890655083678011, 0.010695499707773191, -0.2196725981767179, 0.19647584655458183, 0.13850724251212554, -0.0

[0.0, 0.033050847457627125, 0.07829694793185435, 0.1518842052612546, 0.10243316624895556, 0.19058818703424163, -0.01647514871715118, -0.208907699672711, 0.23137947782912477, 0.046112040258746434, -0.10260869565217395, 0.11556545521661804, 0.10039422838047465, 0.34647637825814587, -0.0764077180607128, -0.14675194300034217, 0.00033316215725418593, -0.18288518932516706, -0.07805762699207162, -0.007890448141556677, 0.004139459515862316, 0.32283653846153887, 0.004849630081334184, -0.07983437380566127, 0.02619726174838396, 0.0034851750211948797, 0.4981735321218243, -0.05, 0.04989596199333189, 0.2422462079398175, 0.1553504866484877, -0.006233250863012257, 0.06550655235306911, -0.18887355298308134, 0.33763197643969034, 0.21609100576842524, 0.1449133754575243, -0.05167262546785113, 0.2595556702833806, 0.024365178709076875, 0.10852140500011201, 0.08765225343850769, -0.059504052178207444, -0.016898012857977502, -0.1793828892005609, 0.21301207672270867, 0.12118365875933472, -0.009433962264150943, 

[0.05, -0.0057593378625489956, 0.04871534664883015, 0.06390083558613913, 0.01920416455248526, 0.19311342894477423, 0.0, -0.21044756360718686, 0.07735523441361809, -0.41093648781861797, -0.09469044564781541, 0.0385411688731024, 0.26621648615664484, 0.0, 0.0, -1.0114550043546078, 0.06929377085850028, -0.019182509505703403, -1.8954534164576398, 0.147448439068617, 0.0051363971491767005, 0.15054945054945051, 0.015, 0.015693503772010144, -0.011538461538461539, 0.31584728196901846, -0.01790125430761692, -0.05, 0.03360583016476553, -0.18988511386401724, 0.36941479757918205, -0.22173138363234215, 0.0, 0.06494765051420683, -0.07959777177029582, 0.10576923076923062, 0.07970529204554766, 0.10513449310698257, 0.13177655677655678, 0.015, -0.43047233648271827, 0.07459565657101172, 0.24633332420149906, 0.1, -0.18486907114624512, -0.02890243902439025, 0.0, 0.1289247311827957, -0.607824412596275, 0.1846876671533686, -0.33413461538461525, 0.2650000000000001, -0.08664347159261053, 0.11240893735771446, 0.0

[0.05, 0.010000000000000002, -0.0649684818285794, -0.041666666666666644, 0.1690924204001129, 0.031047423349772632, 0.006476190476190476, -0.15149561725831015, 0.06688282617298308, -0.28123785382340544, -0.08171547543622383, 0.017529001129247507, 0.2257670161418015, 0.0008623344625808438, 0.11428571428571428, -0.8028922813458201, 0.5665963653070811, -0.03685678073510772, -0.20302034224886073, -0.010630832955571776, 0.0, 0.034498168498168495, 0.015, -0.014400293442866885, -0.01388221153846154, 0.37115252789175096, -0.06093880954935058, 0.0353550295857987, -0.025063371356146766, -0.009695575481003624, 0.3695145518940558, -0.125, -0.03033806353794035, 0.02577483655957596, -0.1668831168831169, -0.07311598124098126, 0.08608695652173914, 0.08225602027883396, -0.2757575757575758, -0.004411764705882354, -0.3472623558870397, -0.09008999197778461, -0.010295296408007885, 0.11250000000000002, -0.12669836956521735, 0.023165084925443852, 0.0, -0.02952380952380952, -0.436845658322213, -0.0126734342548

[0.0, 0.0031457360530842966, -0.029805312288132144, -0.06430736462685332, 0.1314777532643466, 0.09098735344508552, 0.0, -0.14195230316260202, 0.06072367635566657, -0.2289814814814816, -0.09569825802110987, -0.13312657818115356, 0.17433966288877395, 0.0006467508469356329, 0.10999999999999996, -0.12611279794258454, 0.3839620837346558, -0.03666666666666666, -0.21790789334356567, 0.14010070169009745, 0.008650774145981812, 0.06538461538461539, 0.0, 0.03163688226734945, -0.01388221153846154, 0.2706745920503418, -0.039384835660034104, 0.03387573964497034, -0.004999999999998464, -0.009676624576532187, 0.36860278401192537, -0.11949496190710282, -0.01506123996114757, -0.012800142074169856, -0.12357865776396147, -0.014412462537462551, -0.0418693479507483, 0.1638062917474683, -0.17728105228105234, -0.017647058823529415, -0.29207297146766326, 0.11878924996572034, -0.008092037420631203, 0.025, -0.08437499999999998, 0.03189131104296942, 0.0, 0.06999999999999999, -0.3280382483238965, 0.320737426200221

In [ ]:
import pprint
pp = pprint.PrettyPrinter(indent = 4)
pp.pprint(class_to_feat_mat)

In [ ]:
import ast
import re

from string import punctuation
from porter import PorterStemmer

p = PorterStemmer()

stopword_list_file = 'stopword-list'
# test_list_file, test_class_list_file = 'test-list', 'test-class-list'
test_list_file, test_class_list_file = 'train-class-list-copy', 'train-class-list-copy'
model_file = 'model'

with open(model_file, 'r') as m:
    lines = list(map(lambda w: ast.literal_eval(w), m.readlines()))
    class_list, class_to_feat_to_index, class_to_weights = lines

text_to_class = {}
with open(test_class_list_file, 'r') as a:
    lines = map(lambda ln: ln.strip().split(' '), a.readlines())
    for ln in lines:
        file, curr_class = ln
        # text = file.split('/')[-1]
        text = re.split('[(\\\\)(\\)(\/)]', file)[-1]
        text_to_class[text] = curr_class

k = 4
answer_lines = []
score, total = 0, 0
with open(test_list_file, 'r') as t:
    # lines = map(lambda ln: ln.strip(), t.readlines())
    lines = map(lambda ln: ln.strip().split(' ')[0], t.readlines())
    for ln in lines:
        file = ln
        # text = file.split('/')[-1]
        text = re.split('[(\\\\)(\\)(\/)]', file)[-1]
        flat_text, freq_dict = [], {}
        with open(file, 'r') as f:
            processed_lines = map(lambda ln: strip_and_filter_line(ln), f.readlines())
            for line in processed_lines:
                flat_text.extend(list(map(lambda word: p.stem(word, 0, len(word) - 1), line)))
            num_words = len(flat_text)
            last_index = num_words - 1
            for i in range(num_words):
                word = flat_text[i]
                if word not in freq_dict:
                    freq_dict[word] = 1
                else:
                    freq_dict[word] += 1
                if i < last_index:
                    bigram = '{} {}'.format(word, flat_text[i + 1])
                    if bigram not in freq_dict:
                        freq_dict[bigram] = 1
                    else:
                        freq_dict[bigram] += 1
            fin_freq_dict = { word: freq for word, freq in freq_dict.items() if freq >= k }
            compromise = 1
            while not fin_freq_dict:
                fin_freq_dict = { word: freq for word, freq in freq_dict.items() if freq >= k - compromise }
                compromise += 1
            sum_freq = sum(fin_freq_dict.values())
            normalized_freq_dict = { word: freq / sum_freq for word, freq in fin_freq_dict.items() }
            answer = { c: 0 for c in class_list }
            for c in class_list:
                for w in class_to_feat_to_index[c]:
                    if w in normalized_freq_dict:
                        index = class_to_feat_to_index[c][w]
                        answer[c] += class_to_weights[c][index] * normalized_freq_dict[w]
            answer = sorted(answer.items(), key = lambda x: x[1], reverse = True)[0][0]
            answer_lines.append('{} {}\n'.format(file, answer))
            total += 1
            if text_to_class[text] == answer:
                score += 1

with open('answer', 'w') as f:
    f.writelines(answer_lines)

In [ ]:
print(score)
print(total)